# Makine Öğrenimi ile Fiyat Konut Tahmini: Hiperparametre Optimizasyonu

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
df = pd.read_excel(r"C:\Users\Monster\Desktop\konut-fiyat-tahmin-projesi\Veri Seti\istanbul_konut2.xlsx")

In [3]:
# Eksik değerlerin toplamını ve oranını göster
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100

# Eksik değer tablosu oluştur
missing_data = pd.DataFrame({'Eksik Değerler': missing_values, 'Eksik Oranı (%)': missing_percent})
print(missing_data)

               Eksik Değerler  Eksik Oranı (%)
Fiyat                       0         0.000000
İlce                        0         0.000000
Mahalle                     0         0.000000
Metre Kare                  0         0.000000
Oda Sayısı                  2         0.006988
Yaş                        24         0.083852
Bulunduğu Kat             936         3.270212


In [4]:
# 'Yaş' sütunundaki eksik değerleri medyan ile doldur
df['Yaş'] = df['Yaş'].fillna(df['Yaş'].median())
# 'Bulunduğu Kat' sütunundaki eksik değerleri mod ile doldur
df['Bulunduğu Kat'] = df['Bulunduğu Kat'].fillna(df['Bulunduğu Kat'].mode()[0])
# Oda Sayısı sütunundaki eksik değerleri mod (en sık görülen değer) ile doldur
most_frequent_value = df['Oda Sayısı'].mode()[0]
df['Oda Sayısı'] = df['Oda Sayısı'].fillna(most_frequent_value)
print(df.isnull().sum())

Fiyat            0
İlce             0
Mahalle          0
Metre Kare       0
Oda Sayısı       0
Yaş              0
Bulunduğu Kat    0
dtype: int64


In [5]:
df

,Fiyat,İlce,Mahalle,Metre Kare,Oda Sayısı,Yaş,Bulunduğu Kat
0,5000000,Adalar,Burgazada Mah,55,2.0,24.0,0.0
1,7500000,Adalar,Nizam Mah,65,2.0,31.0,0.0
2,4750000,Adalar,Nizam Mah,60,2.0,34.0,0.0
3,7000000,Adalar,MadeMah,75,2.0,35.0,0.0
4,4200000,Adalar,Nizam Mah,51,2.0,38.0,0.0
...,...,...,...,...,...,...,...
28617,4150000,Zeytinburnu,VeliefendMah.,150,6.0,29.0,1.0
28618,4500000,Zeytinburnu,VeliefendMah.,170,6.0,30.0,1.0
28619,17000000,Zeytinburnu,Gökalp Mah.,350,12.0,16.0,0.0
28620,19000000,Zeytinburnu,Beştelsiz Mah.,500,12.0,28.0,0.0


In [6]:
df

,Fiyat,İlce,Mahalle,Metre Kare,Oda Sayısı,Yaş,Bulunduğu Kat
0,5000000,Adalar,Burgazada Mah,55,2.0,24.0,0.0
1,7500000,Adalar,Nizam Mah,65,2.0,31.0,0.0
2,4750000,Adalar,Nizam Mah,60,2.0,34.0,0.0
3,7000000,Adalar,MadeMah,75,2.0,35.0,0.0
4,4200000,Adalar,Nizam Mah,51,2.0,38.0,0.0
...,...,...,...,...,...,...,...
28617,4150000,Zeytinburnu,VeliefendMah.,150,6.0,29.0,1.0
28618,4500000,Zeytinburnu,VeliefendMah.,170,6.0,30.0,1.0
28619,17000000,Zeytinburnu,Gökalp Mah.,350,12.0,16.0,0.0
28620,19000000,Zeytinburnu,Beştelsiz Mah.,500,12.0,28.0,0.0


## Target Encoding (Mean Encoding) 

Her bir ilçe/mahalle, o ilçe/mahalledeki ortalama konut fiyatı ile değiştirilir

In [7]:
# Target Encoding uygulama
def target_encode(df, target_col):
    # İlce için ortalama fiyatları hesapla
    ilce_means = df.groupby('İlce')[target_col].mean()
    df['Ilce_encoded'] = df['İlce'].map(ilce_means)

    # Mahalle için ortalama fiyatları hesapla
    mahalle_means = df.groupby('Mahalle')[target_col].mean()
    df['Mahalle_encoded'] = df['Mahalle'].map(mahalle_means)

    return df

# Target Encoding'i uygula
df_encoded = target_encode(df.copy(), 'Fiyat')

# Özellikler (X) ve hedef değişkeni (y) belirleyelim
X = df_encoded[['Metre Kare', 'Oda Sayısı', 'Yaş', 'Bulunduğu Kat', 'Ilce_encoded', 'Mahalle_encoded']]
y = df_encoded['Fiyat']

# Veriyi eğitim ve test olarak ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest modelini oluştur ve eğit
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Test verisi üzerinde tahmin yap
y_pred = model.predict(X_test)

# R² skorunu hesapla
r2 = r2_score(y_test, y_pred)

# Sonucu yazdır
print("Target Encoding uygulanmış veri seti ile R² değeri (Random Forest):")
print("R² Score:", r2)

Target Encoding uygulanmış veri seti ile R² değeri (Random Forest):
R² Score: 0.8296936501480221


# Outlier Değerleri Temizleme

## Hibrid Yöntem - (IQR + Manuel Veri Temizleme) 

In [8]:
df_new=df_encoded.copy()

In [9]:
df_new=df_encoded[df_encoded["Oda Sayısı"]<8]
df_new=df_encoded[df_encoded["Yaş"]<57]
df_new = df_encoded[(df_encoded["Metre Kare"] < 270) & (df_encoded["Metre Kare"] > 40)]

In [10]:
X_clean = df_new[['Metre Kare', 'Oda Sayısı', 'Yaş', 'Bulunduğu Kat', 'Ilce_encoded', 'Mahalle_encoded']]
y_clean = df_new['Fiyat']

# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, test_size=0.2, random_state=42)

# Modeli eğitme
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Modelin performansını değerlendirme
y_pred = rf_model.predict(X_test)
r2 = r2_score(y_test, y_pred)

print(f'Manuel Temizleme Sonrası R^2 Skoru: {r2:.4f}')
print(f'Manuel Temizleme Sonrası Veri sayısı: {len(X_clean)}')

Manuel Temizleme Sonrası R^2 Skoru: 0.8334
Manuel Temizleme Sonrası Veri sayısı: 27961


In [11]:
df_clean = pd.concat([X_clean, y_clean], axis=1)
# İlçe bazında Fiyat, Metre Kare, Oda Sayısı ve Yaş değişkenleri için ortalama ve standart sapma hesaplama
stats = df_clean.groupby('Ilce_encoded').agg(
    Fiyat_mean=('Fiyat', 'mean'), Fiyat_std=('Fiyat', 'std'),
    MetreKare_mean=('Metre Kare', 'mean'), MetreKare_std=('Metre Kare', 'std'),
    OdaSayisi_mean=('Oda Sayısı', 'mean'), OdaSayisi_std=('Oda Sayısı', 'std'),
    Yas_mean=('Yaş', 'mean'), Yas_std=('Yaş', 'std')
).reset_index()

# Aykırı değer eşiğini hesaplama ve kontrol etme
def is_outlier(row, stats):
    ilce_stats = stats[stats['Ilce_encoded'] == row['Ilce_encoded']].iloc[0]
    
    # Eşik değerler
    fiyat_threshold = ilce_stats['Fiyat_mean'] + 2 * ilce_stats['Fiyat_std'] + 1
    metrekare_threshold = ilce_stats['MetreKare_mean'] + 2 * ilce_stats['MetreKare_std'] + 1
    odasayisi_threshold = ilce_stats['OdaSayisi_mean'] + 2 * ilce_stats['OdaSayisi_std'] + 1
    yas_threshold = ilce_stats['Yas_mean'] + 2 * ilce_stats['Yas_std'] + 1

    # Aykırı değer kontrolü
    if (row['Fiyat'] > fiyat_threshold or 
        row['Metre Kare'] > metrekare_threshold or 
        row['Oda Sayısı'] > odasayisi_threshold or 
        row['Yaş'] > yas_threshold):
        return True
    return False

# Aykırı değerleri filtreleme
filtered_data = df_clean[~df_clean.apply(lambda row: is_outlier(row, stats), axis=1)]

# Sonuç
print(f"Orijinal veri boyutu: {len(df_clean)}")
print(f"İlçe Bazlı Temizlenme sonucu  veri boyutu: {len(filtered_data)}")

Orijinal veri boyutu: 27961
İlçe Bazlı Temizlenme sonucu  veri boyutu: 24973


In [12]:
filtered_data

,Metre Kare,Oda Sayısı,Yaş,Bulunduğu Kat,Ilce_encoded,Mahalle_encoded,Fiyat
0,55,2.0,24.0,0.0,1.047431e+07,1.064654e+07,5000000
1,65,2.0,31.0,0.0,1.047431e+07,1.104514e+07,7500000
2,60,2.0,34.0,0.0,1.047431e+07,1.104514e+07,4750000
3,75,2.0,35.0,0.0,1.047431e+07,9.944185e+06,7000000
4,51,2.0,38.0,0.0,1.047431e+07,1.104514e+07,4200000
...,...,...,...,...,...,...,...
28604,150,5.0,30.0,0.0,4.050061e+06,1.965306e+06,3000000
28607,125,5.0,31.0,0.0,4.050061e+06,4.250833e+06,2650000
28608,150,5.0,31.0,1.0,4.050061e+06,3.729308e+06,4449000
28614,150,6.0,25.0,1.0,4.050061e+06,3.708659e+06,4200000


In [13]:
# df_clean = pd.concat([X_clean, y_clean], axis=1)

In [14]:
# Özellikler ve hedef değişkeni belirleme
X = filtered_data[['Metre Kare', 'Oda Sayısı', 'Yaş', 'Bulunduğu Kat', 'Ilce_encoded', 'Mahalle_encoded']]
y = filtered_data['Fiyat']

# IQR yöntemiyle aykırı değerleri temizleme
Q1 = X.quantile(0.25)
Q3 = X.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 2 * IQR
upper_bound = Q3 + 2 * IQR
mask = ~((X < lower_bound) | (X > upper_bound)).any(axis=1)
X_clean = X[mask]
y_clean = y[mask]

# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, test_size=0.2, random_state=42)

# Modeli eğitme
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Modelin performansını değerlendirme
y_pred = rf_model.predict(X_test)
r2 = r2_score(y_test, y_pred)

print(f'R^2 Skoru: {r2:.4f}')
print(f'IQR sonrası Veri boyutu: {len(X_clean)}')

R^2 Skoru: 0.8932
IQR sonrası Veri boyutu: 24129


In [15]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# df_clean = pd.concat([X_clean, y_clean], axis=1)
# # DataFrame yüklü olduğunu varsayıyoruz: 'df'
# def veri_kesif_analizi(df_clean):
    
#     # İlçe Bazında Fiyat Dağılımı
#     plt.figure(figsize=(15, 6))
#     df_clean.boxplot(column='Fiyat', by='Ilce_encoded', rot=90)
#     plt.title('İlçe Bazında Fiyat Dağılımı')
#     plt.suptitle('')  # Otomatik başlığı kaldırır
#     plt.ylabel('Fiyat')
#     plt.tight_layout()
#     plt.show()
    
#     plt.figure(figsize=(12, 6))
    
#     # 1. Grafik: Mülk Büyüklüğü Dağılımı
#     plt.subplot(1, 2, 1)
#     df_clean['Metre Kare'].hist(bins=30)
#     plt.title('Mülk Büyüklüğü Dağılımı')
#     plt.xlabel('Metrekare')
    
#     # 2. Grafik: Mülk Büyüklüğü Kutu Grafiği
#     plt.subplot(1, 2, 2)
#     sns.boxplot(x=df_clean['Metre Kare'])
#     plt.title('Mülk Büyüklüğü Kutu Grafiği')
    
#     plt.tight_layout()
#     plt.show()
    
#     # Yeni bir figure açarak ikinci grup grafikleri yerleştiriyoruz
#     plt.figure(figsize=(12, 6))
    
#     # 3. Grafik: Oda Sayısı Kutu Grafiği
#     plt.subplot(1, 2, 1)
#     sns.boxplot(x=df_clean['Oda Sayısı'])
#     plt.title('Oda Sayısı Kutu Grafiği')
    
#     # 4. Grafik: Yaş Kutu Grafiği
#     plt.subplot(1, 2, 2)
#     sns.boxplot(x=df_clean['Yaş'])
#     plt.title('Bina Yaşı Kutu Grafiği')
    
#     plt.tight_layout()
#     plt.show()
    
#     # Fiyat ve Metrekare İlişkisi
#     plt.figure(figsize=(10, 6))
#     plt.scatter(df_clean['Metre Kare'], df_clean['Fiyat'], alpha=0.5)
#     plt.title('Fiyat ve Mülk Büyüklüğü İlişkisi')
#     plt.xlabel('Metrekare')
#     plt.ylabel('Fiyat')
#     plt.tight_layout()
#     plt.show()
    

# # Fonksiyonu Çağır
# veri_kesif_analizi(df_clean)

# Makine Öğrenmesi Modelelerinin Kurulması

In [16]:
df_clean = pd.concat([X_clean, y_clean], axis=1)

In [17]:
df_clean

,Metre Kare,Oda Sayısı,Yaş,Bulunduğu Kat,Ilce_encoded,Mahalle_encoded,Fiyat
0,55,2.0,24.0,0.0,1.047431e+07,1.064654e+07,5000000
1,65,2.0,31.0,0.0,1.047431e+07,1.104514e+07,7500000
2,60,2.0,34.0,0.0,1.047431e+07,1.104514e+07,4750000
3,75,2.0,35.0,0.0,1.047431e+07,9.944185e+06,7000000
4,51,2.0,38.0,0.0,1.047431e+07,1.104514e+07,4200000
...,...,...,...,...,...,...,...
28604,150,5.0,30.0,0.0,4.050061e+06,1.965306e+06,3000000
28607,125,5.0,31.0,0.0,4.050061e+06,4.250833e+06,2650000
28608,150,5.0,31.0,1.0,4.050061e+06,3.729308e+06,4449000
28614,150,6.0,25.0,1.0,4.050061e+06,3.708659e+06,4200000


In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
df_clean = pd.concat([X_clean, y_clean], axis=1)
X_train = df_clean[['Metre Kare', 'Oda Sayısı', 'Yaş', 'Bulunduğu Kat', 'Ilce_encoded', 'Mahalle_encoded']]
y_train = df_clean['Fiyat']
# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, test_size=0.2, random_state=42)



# Mean Absolute Percentage Error (MAPE) hesaplayan fonksiyon
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Modelleri tanımlama
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),  # L2 düzenlileştirme parametresi
    "Lasso Regression": Lasso(alpha=0.1),  # L1 düzenlileştirme parametresi
    "ElasticNet Regression": ElasticNet(alpha=0.1, l1_ratio=0.5)  # L1 ve L2 kombinasyonu
}

# Her model için eğitim ve değerlendirme
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)  # Modeli eğitme
    y_pred = model.predict(X_test)  # Test seti üzerinde tahmin
    
    # Performans metrikleri
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)  # MAPE hesaplama
    
    results[name] = {"MSE": mse, "R² Score": r2, "MAPE (%)": mape}

# Sonuçları DataFrame olarak gösterme
results_df = pd.DataFrame(results).T
print(results_df)

                                MSE  R² Score   MAPE (%)
Linear Regression      4.218131e+12  0.768247  38.618741
Ridge Regression       4.218129e+12  0.768247  38.618765
Lasso Regression       4.218131e+12  0.768247  38.618742
ElasticNet Regression  4.216822e+12  0.768319  38.639161


In [19]:
X_train

,Metre Kare,Oda Sayısı,Yaş,Bulunduğu Kat,Ilce_encoded,Mahalle_encoded
24168,98,3.0,6.0,0.0,5.327358e+06,8.861630e+06
14870,100,3.0,0.0,6.0,1.206458e+07,1.353250e+07
23243,176,4.0,5.0,0.0,4.495740e+06,4.836000e+06
16636,150,4.0,35.0,8.0,1.206458e+07,7.880303e+06
10312,100,3.0,3.0,3.0,2.759087e+06,2.895897e+06
...,...,...,...,...,...,...
25710,70,2.0,30.0,0.0,6.467187e+06,5.375862e+06
6160,110,3.0,0.0,4.0,4.709827e+06,5.546995e+06
1011,180,5.0,11.0,0.0,6.383488e+06,4.042614e+06
19048,65,2.0,10.0,0.0,2.596074e+06,4.296346e+06


## Ağaç Tabanlı Modeller
#### ✅ Basic Random Forest
#### ✅ Extra Trees
#### ✅ XGBoost
#### ✅ LightGBM
#### ✅ Deep Gradient Boosting

In [20]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # 0'a bölünmeyi engellemek için küçük bir değer ekliyoruz
    epsilon = 1e-10
    return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

def train_and_evaluate_models(X_train, X_test, y_train, y_test):
    results = {}
    
    # 1. Basit Random Forest
    rf1 = RandomForestRegressor(n_estimators=100, random_state=42)
    rf1.fit(X_train, y_train)
    results['Random Forest'] = rf1
    
    
    # 2. Extra Trees
    et = ExtraTreesRegressor(
        n_estimators=100,
        random_state=42
    )
    et.fit(X_train, y_train)
    results['Extra Trees'] = et
    
    # 3. Gradient Boosting with Deep Trees
    gb_deep = GradientBoostingRegressor(
        n_estimators=100,
        max_depth=8,
        learning_rate=0.05,
        subsample=0.8,
        random_state=42
    )
    
    # 4. XGBoost
    xgb = XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        random_state=42
    )
    xgb.fit(X_train, y_train)
    results['XGBoost'] = xgb
    
    # 5. LightGBM
    lgbm = LGBMRegressor(
        n_estimators=100,
        learning_rate=0.1,
        random_state=42
    )
    lgbm.fit(X_train, y_train)
    results['LightGBM'] = lgbm
    
     # 6. Gradient Boosting with Deep Trees
    gb_deep = GradientBoostingRegressor(
        n_estimators=100,
        max_depth=8,
        learning_rate=0.05,
        subsample=0.8,
        random_state=42
    )
    gb_deep.fit(X_train, y_train)
    results['Deep Gradient Boosting'] = gb_deep
    # Modelleri değerlendir
    print("Model Performansları:")
    print("-" * 80)
    print(f"{'Model':30} {'MSE':>10} {'R2':>15} {'MAPE':>13}")
    print("-" * 80)
    
    for name, model in results.items():
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)
        
        print(f"{name:30} {mse:10.4f} {r2:10.4f} {mape:10.2f}%")
    
    print("-" * 80)
    return results

# Kullanım örneği:
results = train_and_evaluate_models(X_train, X_test, y_train, y_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001213 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 528
[LightGBM] [Info] Number of data points in the train set: 19303, number of used features: 6
[LightGBM] [Info] Start training from score 5578966.878724
Model Performansları:
--------------------------------------------------------------------------------
Model                                 MSE              R2          MAPE
--------------------------------------------------------------------------------
Random Forest                  1943942434180.6855     0.8932      17.07%
Extra Trees                    2004679494114.4153     0.8899      17.02%
XGBoost                        2081732820992.0000     0.8856      20.24%
LightGBM                       2064

In [21]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # 0'a bölünmeyi engellemek için küçük bir değer ekliyoruz
    epsilon = 1e-10
    return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

def train_and_evaluate_models(X_train, X_test, y_train, y_test):
    results = {}
    
    # 1. Basit Random Forest
    rf1 = RandomForestRegressor(n_estimators=100, random_state=42)
    rf1.fit(X_train, y_train)
    results['Random Forest'] = rf1
    
    # 2. Extra Trees
    et = ExtraTreesRegressor(
        n_estimators=100,
        random_state=42
    )
    et.fit(X_train, y_train)
    results['Extra Trees'] = et
    
    # 3. Gradient Boosting with Deep Trees
    gb_deep = GradientBoostingRegressor(
        n_estimators=100,
        max_depth=8,
        learning_rate=0.05,
        subsample=0.8,
        random_state=42
    )
    gb_deep.fit(X_train, y_train)
    results['Deep Gradient Boosting'] = gb_deep
    
    # 4. XGBoost
    xgb = XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        random_state=42
    )
    xgb.fit(X_train, y_train)
    results['XGBoost'] = xgb
    
    # 5. LightGBM
    lgbm = LGBMRegressor(
        n_estimators=100,
        learning_rate=0.1,
        random_state=42
    )
    lgbm.fit(X_train, y_train)
    results['LightGBM'] = lgbm
    
    # Modelleri değerlendir
    print("Model Performansları:")
    print("-" * 120)
    print(f"{'Model':30} {'Train MSE':>10} {'Train R2':>15} {'Train MAPE':>13} {'Test MSE':>10} {'Test R2':>15} {'Test MAPE':>13}")
    print("-" * 120)
    
    for name, model in results.items():
        # Eğitim ve test seti üzerindeki tahminler
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        
        # Eğitim seti metrikleri
        train_mse = mean_squared_error(y_train, y_train_pred)
        train_r2 = r2_score(y_train, y_train_pred)
        train_mape = mean_absolute_percentage_error(y_train, y_train_pred)
        
        # Test seti metrikleri
        test_mse = mean_squared_error(y_test, y_test_pred)
        test_r2 = r2_score(y_test, y_test_pred)
        test_mape = mean_absolute_percentage_error(y_test, y_test_pred)
        
        # Sonuçları yazdır
        print(f"{name:30} {train_mse:10.4f} {train_r2:10.4f} {train_mape:10.2f}% {test_mse:10.4f} {test_r2:10.4f} {test_mape:10.2f}%")
    
    print("-" * 120)
    return results

# Kullanım örneği:
results = train_and_evaluate_models(X_train, X_test, y_train, y_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001025 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 528
[LightGBM] [Info] Number of data points in the train set: 19303, number of used features: 6
[LightGBM] [Info] Start training from score 5578966.878724
Model Performansları:
------------------------------------------------------------------------------------------------------------------------
Model                           Train MSE        Train R2    Train MAPE   Test MSE         Test R2     Test MAPE
------------------------------------------------------------------------------------------------------------------------
Random Forest                  426240863671.8631     0.9760       7.28% 1943942434180.6855     0.8932      17.07%
Extra Trees        

## Random Forest Hiperparametre Optimizasyonu

In [22]:
from sklearn.model_selection import GridSearchCV

# Parametrelerin belirlenmesi
param_grid = {
    'n_estimators': [100, 200],  # Ağaç sayısı
    'max_depth': [10, 20],  # Ağaç derinliği
    'min_samples_split': [2, 5],  # Düğüm bölünmesi için minimum örnek sayısı
    'min_samples_leaf': [ 4, 5],  # Yaprak düğümdeki minimum örnek sayısı
    'max_features': ['sqrt', 'log2', None],  # Geçerli max_features değerleri
    'bootstrap': [ False]  # Bootstrap örnekleme
}

# RandomForestRegressor için GridSearchCV tanımlama
rf = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Parametre optimizasyonu yapmak
grid_search.fit(X_train, y_train)

# En iyi parametreler ve model
print(f"En İyi Parametreler: {grid_search.best_params_}")
print(f"En İyi Model Skoru (MSE): {-grid_search.best_score_}")

# En iyi modeli alalım
best_rf_model = grid_search.best_estimator_

# En iyi modeli test seti ile değerlendirelim
y_pred_best_rf = best_rf_model.predict(X_test)
mse_best_rf = mean_squared_error(y_test, y_pred_best_rf)
r2_best_rf = r2_score(y_test, y_pred_best_rf)
mape_best_rf = mean_absolute_percentage_error(y_test, y_pred_best_rf)

print(f"Best Random Forest Model Performance on Test Set:")
print(f"MSE: {mse_best_rf:.4f}")
print(f"R²: {r2_best_rf:.4f}")
print(f"MAPE: {mape_best_rf:.2f}%")

# Parametre sonuçlarını görme
cv_results = grid_search.cv_results_

# cv_results'ü pandas DataFrame olarak göstermek
results_df = pd.DataFrame(cv_results)

# İlgili parametrelerin ve performans skorlarının dökümünü görmek
print("\nTüm Parametre Kombinasyonları ve Sonuçları:")
print(results_df[['param_n_estimators', 'param_max_depth', 'param_min_samples_split', 
                  'param_min_samples_leaf', 'param_max_features', 'param_bootstrap', 'mean_test_score']])

Fitting 3 folds for each of 48 candidates, totalling 144 fits
En İyi Parametreler: {'bootstrap': False, 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}
En İyi Model Skoru (MSE): 2274683530224.4297
Best Random Forest Model Performance on Test Set:
MSE: 1930054814800.8032
R²: 0.8940
MAPE: 17.89%

Tüm Parametre Kombinasyonları ve Sonuçları:
    param_n_estimators  param_max_depth  param_min_samples_split  \
0                  100               10                        2   
1                  200               10                        2   
2                  100               10                        5   
3                  200               10                        5   
4                  100               10                        2   
5                  200               10                        2   
6                  100               10                        5   
7                  200               10                

## RF için Overfitting Kontrolü 

In [23]:
# Modeli en iyi parametrelerle oluştur
best_params = {'n_estimators': 200, 'max_depth': 30, 'min_samples_split': 2, 
               'min_samples_leaf': 5, 'max_features': 'sqrt', 'bootstrap': False}

# Optuna sonuçları
#'n_estimators': 222, 'max_depth': 30, 'min_samples_split': 7, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'bootstrap': False}

rf = RandomForestRegressor(**best_params, random_state=42)

# Modeli eğit
rf.fit(X_train, y_train)

# Eğitim seti için tahmin yap
y_train_pred = rf.predict(X_train)

# Test seti için tahmin yap
y_pred = rf.predict(X_test)

# Performans metriklerini hesapla
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

# Sonuçları yazdır
print(f"Eğitim Seti R² Skoru: {r2_train:.4f}")
print(f"Test Seti R² Skoru: {r2_test:.4f}")
print(f"Test Seti MAPE: {mape:.4f}")
print(f"Test Seti MSE: {mse:.4f}")

Eğitim Seti R² Skoru: 0.9271
Test Seti R² Skoru: 0.8919
Test Seti MAPE: 18.1607
Test Seti MSE: 1966991376172.9995


## Extra Trees için Hiperparametre Optimizasyonu

In [24]:
# Parametrelerin belirlenmesi (Extra Trees için optimize edilmiş)
param_grid = {
    'n_estimators': [100, 200],  # Ağaç sayısı
    'max_depth': [20, None],  # Ağaç derinliği
    'min_samples_split': [4, 5],  # Düğüm bölünmesi için minimum örnek sayısı
    'min_samples_leaf': [1, 2],  # Yaprak düğümdeki minimum örnek sayısı
    'max_features': ['sqrt', 'log2', None]  # Geçerli max_features değerleri
}

# ExtraTreesRegressor için GridSearchCV tanımlama
et = ExtraTreesRegressor(random_state=42)

grid_search = GridSearchCV(estimator=et, param_grid=param_grid, 
                           cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Parametre optimizasyonu yapmak
grid_search.fit(X_train, y_train)

# En iyi parametreler ve model
print(f"En İyi Parametreler: {grid_search.best_params_}")
print(f"En İyi Model Skoru (MSE): {-grid_search.best_score_}")

# En iyi modeli alalım
best_et_model = grid_search.best_estimator_

# En iyi modeli test seti ile değerlendirelim
y_pred_best_et = best_et_model.predict(X_test)
mse_best_et = mean_squared_error(y_test, y_pred_best_et)
r2_best_et = r2_score(y_test, y_pred_best_et)
mape_best_et = mean_absolute_percentage_error(y_test, y_pred_best_et)

print(f"Best Extra Trees Model Performance on Test Set:")
print(f"MSE: {mse_best_et:.4f}")
print(f"R²: {r2_best_et:.4f}")
print(f"MAPE: {mape_best_et:.2f}%")

# Parametre sonuçlarını görme
cv_results = grid_search.cv_results_

# cv_results'ü pandas DataFrame olarak göstermek
results_df = pd.DataFrame(cv_results)

# İlgili parametrelerin ve performans skorlarının dökümünü görmek
print("\nTüm Parametre Kombinasyonları ve Sonuçları:")
print(results_df[['param_n_estimators', 'param_max_depth', 'param_min_samples_split', 
                  'param_min_samples_leaf', 'param_max_features', 'mean_test_score']])

Fitting 3 folds for each of 48 candidates, totalling 144 fits
En İyi Parametreler: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 200}
En İyi Model Skoru (MSE): 2188044597450.5989
Best Extra Trees Model Performance on Test Set:
MSE: 1856074995856.7344
R²: 0.8980
MAPE: 17.10%

Tüm Parametre Kombinasyonları ve Sonuçları:
    param_n_estimators param_max_depth  param_min_samples_split  \
0                  100              20                        4   
1                  200              20                        4   
2                  100              20                        5   
3                  200              20                        5   
4                  100              20                        4   
5                  200              20                        4   
6                  100              20                        5   
7                  200              20                        5   
8               

In [26]:
# MAPE hesaplama fonksiyonu
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    epsilon = 1e-10  # 0'a bölünmeyi engellemek için küçük bir değer
    return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

# En iyi parametrelerle model oluştur
best_params = {'max_depth': 20, 'max_features': None, 'min_samples_leaf': 3, 
               'min_samples_split': 7, 'n_estimators': 200}

# Extra Trees modelini oluştur
model = ExtraTreesRegressor(**best_params, random_state=42)

# Modeli eğit
model.fit(X_train, y_train)

# Test seti üzerinde tahmin yap
y_pred_test = model.predict(X_test)

# Eğitim seti üzerinde tahmin yap
y_pred_train = model.predict(X_train)

# Performans metriklerini hesapla (Test Seti)
mse_test = mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)
mape_test = mean_absolute_percentage_error(y_test, y_pred_test)

# Performans metriklerini hesapla (Eğitim Seti)
mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)
mape_train = mean_absolute_percentage_error(y_train, y_pred_train)

# Sonuçları yazdır
print(f"Test Seti MSE: {mse_test:.4f}")
print(f"Test Seti R²: {r2_test:.4f}")
print(f"Test Seti MAPE: {mape_test:.2f}%")

print("-" * 50)

print(f"Eğitim Seti MSE: {mse_train:.4f}")
print(f"Eğitim Seti R²: {r2_train:.4f}")
print(f"Eğitim Seti MAPE: {mape_train:.2f}%")

Test Seti MSE: 1926184328463.9543
Test Seti R²: 0.8942
Test Seti MAPE: 17.62%
--------------------------------------------------
Eğitim Seti MSE: 1072354844463.1097
Eğitim Seti R²: 0.9395
Eğitim Seti MAPE: 12.57%


## Deep Gradient Boosting (GradientBoostingRegressor) için Hiperparametre Optimizasyonu 

In [ ]:
# Parametrelerin belirlenmesi
param_grid = {
    'n_estimators': [100, 200],  # Ağaç sayısı
    'max_depth': [3, 5],  # Ağaç derinliği
    'learning_rate': [0.01, 0.05],  # Öğrenme hızı
    'subsample': [0.8, 1.0],  # Alt örnekleme oranı
    'min_samples_split': [2, 5],  # Düğüm bölünmesi için minimum örnek sayısı
    'min_samples_leaf': [1, 2, 4]  # Yaprak düğümdeki minimum örnek sayısı
}

# GradientBoostingRegressor için GridSearchCV tanımlama
gb = GradientBoostingRegressor(random_state=42)

grid_search = GridSearchCV(estimator=gb, param_grid=param_grid, 
                           cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Parametre optimizasyonu yapmak
grid_search.fit(X_train, y_train)

# En iyi parametreler ve model
print(f"En İyi Parametreler: {grid_search.best_params_}")
print(f"En İyi Model Skoru (MSE): {-grid_search.best_score_}")

# En iyi modeli alalım
best_gb_model = grid_search.best_estimator_

# En iyi modeli test seti ile değerlendirelim
y_pred_best_gb = best_gb_model.predict(X_test)
mse_best_gb = mean_squared_error(y_test, y_pred_best_gb)
r2_best_gb = r2_score(y_test, y_pred_best_gb)
mape_best_gb = mean_absolute_percentage_error(y_test, y_pred_best_gb)

print(f"Best Gradient Boosting Model Performance on Test Set:")
print(f"MSE: {mse_best_gb:.4f}")
print(f"R²: {r2_best_gb:.4f}")
print(f"MAPE: {mape_best_gb:.2f}%")

# Parametre sonuçlarını görme
cv_results = grid_search.cv_results_

# cv_results'ü pandas DataFrame olarak göstermek
results_df = pd.DataFrame(cv_results)

# İlgili parametrelerin ve performans skorlarının dökümünü görmek
print("\nTüm Parametre Kombinasyonları ve Sonuçları:")
print(results_df[['param_n_estimators', 'param_max_depth', 'param_learning_rate', 
                  'param_subsample', 'param_min_samples_split', 'param_min_samples_leaf', 'mean_test_score']])

Fitting 3 folds for each of 96 candidates, totalling 288 fits
